# WB

## Instalacja bibliotek

In [2]:
!pip install h5py


In [8]:
!pip install keras

In [20]:
!python -m pip install --upgrade pip

  Using cached pip-23.1-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 20.2.3
    Uninstalling pip-20.2.3:
      Successfully uninstalled pip-20.2.3


In [22]:
!pip install grpcio>=1.48.2


In [27]:
!pip install scikit-learn

In [5]:
!pip install tensorflow --user

  Using cached tensorflow-2.12.0-cp39-cp39-win_amd64.whl (1.9 kB)
  Using cached tensorflow_intel-2.12.0-cp39-cp39-win_amd64.whl (272.8 MB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached libclang-16.0.0-py2.py3-none-win_amd64.whl (24.4 MB)
  Using cached termcolor-2.3.0-py3-none-any.whl (6.9 kB)
  Using cached keras-2.12.0-py2.py3-none-any.whl (1.7 MB)
  Using cached tensorflow_estimator-2.12.0-py2.py3-none-any.whl (440 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached tensorboard-2.12.2-py3-none-any.whl (5.6 MB)
  Using cached protobuf-4.22.3-cp39-cp39-win_amd64.whl (420 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached jax-0.4.8-py3-none-any.whl
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using cached flatbuffers-23.3.3-py2.py3-none-any.whl (26 kB)
  Using cached wrapt-1.14.1-cp39-cp39-win_amd64.whl (35 kB)
  Using cached tensorflow_io_gc

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [5]:
#rozpakuj zip
import gzip
import shutil

input_file = 'camelyonpatch_level_2_split_test_x.h5.gz'
output_file = 'camelyonpatch_level_2_split_test_x.h5'

with gzip.open(input_file, 'rb') as f_in:
    with open(output_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
        
        
input_file = 'camelyonpatch_level_2_split_test_y.h5.gz'
output_file = 'camelyonpatch_level_2_split_test_y.h5'

with gzip.open(input_file, 'rb') as f_in:
    with open(output_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [1]:
import h5py

from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import h5py
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
import keras
from tensorflow.keras.callbacks import Callback
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score 
from scipy.stats import mode

## Przygotowanie zbioru danych

In [2]:
#funkcja do podziału danych, ratio_1_0 = ile procent 0 w zbiorze
np.random.seed(243)
def split_data(x, y, train_ratio, val_ratio, test_ratio,ratio_1_0=0.85):
    class_0_indices = np.where(y == 0)[0]
    class_1_indices = np.where(y == 1)[0]
    
    total_samples = len(x)
    class_0_size = len(class_0_indices)

    class_1_size = int((class_0_size/ratio_1_0)) - class_0_size
   
    np.random.shuffle(class_0_indices)
    np.random.shuffle(class_1_indices)

    train_size_0 = int(class_0_size * train_ratio)
    val_size_0 = int(class_0_size * val_ratio)
    train_size_1 = int(class_1_size * train_ratio)
    val_size_1 = int(class_1_size * val_ratio)
    test_size_1 = int(class_1_size * test_ratio)
    test_size_0 = int(class_0_size * test_ratio)
    
    x_train = np.concatenate([x[class_0_indices[:train_size_0]], x[class_1_indices[:train_size_1]]])
    y_train = np.concatenate([y[class_0_indices[:train_size_0]], y[class_1_indices[:train_size_1]]])
    x_val = np.concatenate([x[class_0_indices[train_size_0:train_size_0 + val_size_0]], x[class_1_indices[train_size_1:train_size_1 + val_size_1]]])
    y_val = np.concatenate([y[class_0_indices[train_size_0:train_size_0 + val_size_0]], y[class_1_indices[train_size_1:train_size_1 + val_size_1]]])
    x_test = np.concatenate([x[class_0_indices[train_size_0 + val_size_0:train_size_0 + val_size_0+test_size_0]], x[class_1_indices[train_size_1 + val_size_1:test_size_1+train_size_1 + val_size_1]]])
    y_test = np.concatenate([y[class_0_indices[train_size_0 + val_size_0:train_size_0 + val_size_0+test_size_0]], y[class_1_indices[train_size_1 + val_size_1:test_size_1+train_size_1 + val_size_1]]])

    train_indices = np.arange(len(x_train))
    val_indices = np.arange(len(x_val))
    test_indices = np.arange(len(x_test))
    np.random.shuffle(train_indices)
    np.random.shuffle(val_indices)
    np.random.shuffle(test_indices)

    x_train, y_train = x_train[train_indices], y_train[train_indices]
    x_val, y_val = x_val[val_indices], y_val[val_indices]
    x_test, y_test = x_test[test_indices], y_test[test_indices]

    return (x_train, y_train), (x_val, y_val), (x_test, y_test)


with h5py.File('camelyonpatch_level_2_split_test_x.h5', 'r') as f:
    x_data = f['x'][:]

with h5py.File('camelyonpatch_level_2_split_test_y.h5', 'r') as f:
    y_data = f['y'][:]

(x_train, y_train), (x_val,y_val) , (x_test, y_test) = split_data(x_data, y_data, 0.3, 0.1,0.3,0.5)

datagen = ImageDataGenerator(
    preprocessing_function=lambda x: x / 255.,
    width_shift_range=4,  
    height_shift_range=4,  
    horizontal_flip=True,  
    vertical_flip=True)  

In [15]:
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print("x_val shape:", x_val.shape)
print("y_val shape:", y_val.shape)
print("x_test shape:", x_test.shape)
print("y_test shape:", y_test.shape)
n_w = len(y_train)
n_1 =y_train.sum()

print("Ilość obserwacji w train:", n_w)
print("Ilość 1 w train:", n_1)
n_w = len(y_val)
n_1 =y_val.sum()
print("Ilość obserwacji w walidacji:", n_w)
print("Ilość 1 w walidacji:", n_1)
n_w = len(y_test)
n_1 =y_test.sum()
print("Ilość obserwacji w test:", n_w)
print("Ilość 1 w test:", n_1)


x_train shape: (9834, 96, 96, 3)
y_train shape: (9834, 1, 1, 1)
x_val shape: (3278, 96, 96, 3)
y_val shape: (3278, 1, 1, 1)
x_test shape: (9834, 96, 96, 3)
y_test shape: (9834, 1, 1, 1)
Ilość obserwacji w train: 9834
Ilość 1 w train: 4917
Ilość obserwacji w walidacji: 3278
Ilość 1 w walidacji: 1639
Ilość obserwacji w test: 9834
Ilość 1 w test: 4917


In [16]:
y_test = np.reshape(y_test, (y_test.shape[0], 1))
y_train = np.reshape(y_train, (y_train.shape[0], 1))
y_val = np.reshape(y_val, (y_val.shape[0], 1))

## Trening modelu

In [17]:
from tensorflow.keras.callbacks import Callback

class MetricsCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        val_preds = self.model.predict(x_val)
        val_preds = val_preds.astype(int)
        #val_preds = np.argmax(val_preds, axis=1)

        val_accuracy = accuracy_score(y_val, val_preds)
        val_precision = precision_score(y_val, val_preds)
        val_recall = recall_score(y_val, val_preds)
        val_f1_score = f1_score(y_val, val_preds)

        print(f"Validation accuracy: {val_accuracy:.4f}")
        print(f"Validation precision: {val_precision:.4f}")
        print(f"Validation recall: {val_recall:.4f}")
        print(f"Validation F1-score: {val_f1_score:.4f}\n")

metrics_callback = MetricsCallback()

#y_val_one_hot = keras.utils.to_categorical(y_val, num_classes=2)

In [18]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(96, 96, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#y_train_one_hot = keras.utils.to_categorical(y_train, num_classes=2)

model.fit(datagen.flow(x_train, y_train, batch_size=256),
                    steps_per_epoch=len(x_train) // 256,
                    epochs=3,

                    callbacks=[metrics_callback],
                    validation_data=(x_val, y_val)
         )


Epoch 1/3
103/103 [==============================] - 3s 25ms/step
Validation accuracy: 0.7663
Validation precision: 0.7160
Validation recall: 0.8829
Validation F1-score: 0.7907

38/38 [==============================] - 36s 938ms/step - loss: 0.6335 - accuracy: 0.6726 - val_loss: 65.7379 - val_accuracy: 0.7599
Epoch 2/3
103/103 [==============================] - 3s 25ms/step
Validation accuracy: 0.7441
Validation precision: 0.6869
Validation recall: 0.8969
Validation F1-score: 0.7780

38/38 [==============================] - 36s 947ms/step - loss: 0.4871 - accuracy: 0.7824 - val_loss: 73.3671 - val_accuracy: 0.7389
Epoch 3/3
103/103 [==============================] - 3s 26ms/step
Validation accuracy: 0.7642
Validation precision: 0.7123
Validation recall: 0.8865
Validation F1-score: 0.7899

38/38 [==============================] - 36s 950ms/step - loss: 0.4824 - accuracy: 0.7807 - val_loss: 64.7252 - val_accuracy: 0.7602


In [20]:
#Zapisz model
model.save('model_sigmoid_1.h5')

In [3]:
from tensorflow.keras.models import load_model

#Wczytaj model
model = load_model('model_sigmoid_1.h5')

In [19]:
val_preds = model.predict(x_test)
#val_preds = np.argmax(val_preds, axis=1)
val_preds = val_preds.astype(int)

val_accuracy = accuracy_score(y_test, val_preds)
val_precision = precision_score(y_test, val_preds)
val_recall = recall_score(y_test, val_preds)
val_f1_score = f1_score(y_test, val_preds)

print(f"Test accuracy: {val_accuracy:.4f}")
print(f"Test precision: {val_precision:.4f}")
print(f"Test recall: {val_recall:.4f}")
print(f"Test F1-score: {val_f1_score:.4f}\n")

308/308 [==============================] - 8s 24ms/step
Test accuracy: 0.7673
Test precision: 0.7129
Test recall: 0.8951
Test F1-score: 0.7937



In [ ]:
#Konkurencyjny model można sprawdzić !TODO
#image net

## Stworzenie danych tabularycznych z warstw modelu

In [4]:
#Wyekstrachowanie warstwy przedostatniej oraz warstwy ostatniej konwolucyjnej
def get_features(model, data):
    feature_extractor = keras.Model(inputs=model.inputs, outputs=model.layers[-2].output)
    feature_extractor1 = keras.Model(inputs=model.inputs, outputs=model.layers[-4].output)

    return (feature_extractor.predict(data), feature_extractor1.predict(data))

x_test_features,x_test_features_conv = get_features(model, x_test)
x_val_features,x_val_features_conv = get_features(model, x_val)

103/103 [==============================] - 2s 23ms/step


## Anomaly Detection do poprawy jakości modelu

In [22]:
# Train Isolation Forest, Local Outlier Factor, One-Class SVM na Val 
#współczynnik (0.2) zależny od błędu na modelu (u nas daje 75% acc czyli 1-0.75=0.25)
iso_forest = IsolationForest(contamination=0.25)
lof = LocalOutlierFactor(contamination=0.25, novelty=True)
one_class_svm = OneClassSVM(nu=0.25)

iso_forest.fit(x_test_features)
lof.fit(x_test_features)
one_class_svm.fit(x_test_features)

#Predictions dla val
iso_forest_pred = iso_forest.predict(x_val_features)
lof_pred = lof.predict(x_val_features)
one_class_svm_pred = one_class_svm.predict(x_val_features)

#Połączony model voting
combined_pred = mode(np.vstack((iso_forest_pred, lof_pred, one_class_svm_pred)), axis=0)[0].flatten()

models = [iso_forest_pred, lof_pred, one_class_svm_pred, combined_pred]
model_names = ['Isolation Forest', 'Local Outlier Factor', 'One-Class SVM', 'Combined (Voting)']


  4/308 [..............................] - ETA: 7s 

C:\Users\HP\AppData\Local\Temp\ipykernel_22032\2980966709.py:17: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  combined_pred = mode(np.vstack((iso_forest_pred, lof_pred, one_class_svm_pred)), axis=0)[0].flatten()


308/308 [==============================] - 7s 23ms/step


In [25]:
#Wyniki
model_preds = np.round(model.predict(x_val)).flatten() 
for name, pred in zip(model_names, models):
    print(f'\n{name} Results:')

    # Convert anomalies to binary labels (1 for normal, -1 for anomaly)
    binary_pred = (pred == -1).astype(int)
    binary_y_test = y_val.flatten() 

    incorrect_pred = model_preds != binary_y_test
    
    print('Comparison between incorrect model predictions and detected anomalies')
      
    print('Recall (Wrong labels detected / All wrong labels):', recall_score(incorrect_pred,binary_pred))
    print('Precision (Wrong labels detected / All anomalies detected):', precision_score(incorrect_pred,binary_pred))
    print('Accuracy:',accuracy_score(incorrect_pred,binary_pred))
    print('Comparison between y_val and detected anomalies')
      
    print('Recall:', recall_score(y_val,binary_pred))
    print('Precision:', precision_score(y_val,binary_pred))
    print('Accuracy:',accuracy_score(y_val,binary_pred))
    print('')


103/103 [==============================] - 2s 24ms/step

Isolation Forest Results:
Comparison between incorrect model predictions and detected anomalies
Recall (Wrong labels detected / All wrong labels): 0.15267175572519084
Precision (Wrong labels detected / All anomalies detected): 0.14869888475836432
Accuracy: 0.587248322147651
Comparison between y_val and detected anomalies
Recall: 0.1781574130567419
Precision: 0.3618339529120198
Accuracy: 0.4319707138499085


Local Outlier Factor Results:
Comparison between incorrect model predictions and detected anomalies
Recall (Wrong labels detected / All wrong labels): 0.32697201017811706
Precision (Wrong labels detected / All anomalies detected): 0.30414201183431955
Accuracy: 0.6592434411226358
Comparison between y_val and detected anomalies
Recall: 0.23550945698596706
Precision: 0.45680473372781066
Accuracy: 0.4777303233679073


One-Class SVM Results:
Comparison between incorrect model predictions and detected anomalies
Recall (Wrong labels 

## Anomaly detection do rozpoznawania raka z warstwy konwolucyjnej

In [5]:
#Otrzymanie 15% 1 (raka) jeśli utworzony zbiór miał inaczej (ja stworzyłem 50/50 rak nie rak więc musiałem to odpalić)
#x_val_features_conv
class_1_indices = np.where(y_val == 1)[0]
class_0_indices = np.where(y_val == 0)[0]

np.random.shuffle(class_1_indices)
val_size_1 = int(len(class_1_indices) * 0.2)
    
x_val_features_conv_with_ratio = np.concatenate([x_val_features_conv[class_0_indices], x_val_features_conv[class_1_indices[:val_size_1]]])
y_val_with_ratio = np.concatenate([y_val[class_0_indices], y_val[class_1_indices[:val_size_1]]])
val_indices = np.arange(len(x_val_features_conv_with_ratio))
np.random.shuffle(val_indices)

x_val_features_conv_with_ratio = x_val_features_conv_with_ratio[val_indices]
y_val_with_ratio = y_val_with_ratio[val_indices]
#x_test_features_conv
class_1_indices = np.where(y_test == 1)[0]
class_0_indices = np.where(y_test == 0)[0]

np.random.shuffle(class_1_indices)
test_size_1 = int(len(class_1_indices) * 0.2)
    
x_test_features_conv_with_ratio = np.concatenate([x_test_features_conv[class_0_indices], x_test_features_conv[class_1_indices[:test_size_1]]])
y_test_with_ratio = np.concatenate([y_test[class_0_indices], y_test[class_1_indices[:test_size_1]]])
test_indices = np.arange(len(x_test_features_conv_with_ratio))
np.random.shuffle(test_indices)

x_test_features_conv_with_ratio = x_test_features_conv_with_ratio[test_indices]
y_test_with_ratio = y_test_with_ratio[test_indices]

In [27]:
print('Ratio:',y_test_with_ratio.sum()/len(y_test_with_ratio))

Ratio: 0.16661016949152543


In [6]:
# Train Isolation Forest, Local Outlier Factor, One-Class SVM na Val 
iso_forest = IsolationForest(contamination=0.16)
lof = LocalOutlierFactor(contamination=0.16, novelty=True)
one_class_svm = OneClassSVM(nu=0.16)

iso_forest.fit(x_test_features_conv_with_ratio)
lof.fit(x_test_features_conv_with_ratio)
one_class_svm.fit(x_test_features_conv_with_ratio)

#Predictions dla Test
iso_forest_pred = iso_forest.predict(x_val_features_conv_with_ratio)
lof_pred = lof.predict(x_val_features_conv_with_ratio)
one_class_svm_pred = one_class_svm.predict(x_val_features_conv_with_ratio)

#Połączony model voting
combined_pred = mode(np.vstack((iso_forest_pred, lof_pred, one_class_svm_pred)), axis=0)[0].flatten()

C:\Users\HP\AppData\Local\Temp\ipykernel_12720\68301960.py:16: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  combined_pred = mode(np.vstack((iso_forest_pred, lof_pred, one_class_svm_pred)), axis=0)[0].flatten()


In [7]:
#Wyniki
models = [iso_forest_pred, lof_pred, one_class_svm_pred, combined_pred]
model_names = ['Isolation Forest', 'Local Outlier Factor', 'One-Class SVM', 'Combined (Voting)']
for name, pred in zip(model_names, models):
    print(f'\n{name} Results:')

    # Convert anomalies to binary labels (1 for normal, -1 for anomaly)
    binary_pred = (pred == -1).astype(int)
    binary_y_test = y_val_with_ratio.flatten()
    
    print('Comparison between detected anomalies and y_test')
      
    print('Recall:', recall_score(binary_y_test,binary_pred))
    print('Precision:', precision_score(binary_y_test,binary_pred))
    print('Accuracy:',accuracy_score(binary_y_test,binary_pred))
    


Isolation Forest Results:
Comparison between detected anomalies and y_test
Recall: 0.04281345565749235
Precision: 0.04590163934426229
Accuracy: 0.6927772126144456

Local Outlier Factor Results:
Comparison between detected anomalies and y_test
Recall: 0.024464831804281346
Precision: 0.024691358024691357
Accuracy: 0.6770091556459817

One-Class SVM Results:
Comparison between detected anomalies and y_test
Recall: 0.0672782874617737
Precision: 0.07213114754098361
Accuracy: 0.7009155645981688

Combined (Voting) Results:
Comparison between detected anomalies and y_test
Recall: 0.024464831804281346
Precision: 0.03347280334728033
Accuracy: 0.7202441505595117


## Wytrenowanie Autoenkodera

In [ ]:
#todo zrób lepszy autoenkoder

In [8]:
(x_train, y_train), (x_val,y_val) , (x_test, y_test) = split_data(x_data, y_data, 0.3, 0.1,0.3,0.8)

In [9]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Dense, Reshape
from tensorflow.keras.models import Model

input_shape = (96, 96, 3)

# Encoder
input_img = Input(shape=input_shape)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Flatten()(x)
encoded = Dense(30, activation='relu')(x)

# Decoder
x = Dense(24 * 24 * 64, activation='relu')(encoded)
x = Reshape((24, 24, 64))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

# Autoencoder model
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

# Train the autoencoder
autoencoder.fit(x_train, x_train,
                epochs=5,
                batch_size=256,
                shuffle=True,
                validation_split=0.2)

# Encoder model to extract features
encoder = Model(input_img, encoded)


Epoch 1/5
20/20 [==============================] - 49s 2s/step - loss: 31963.3320 - val_loss: 31321.9570
Epoch 2/5
20/20 [==============================] - 48s 2s/step - loss: 31952.0156 - val_loss: 31321.9570
Epoch 3/5
20/20 [==============================] - 48s 2s/step - loss: 31952.0156 - val_loss: 31321.9570
Epoch 4/5
20/20 [==============================] - 50s 3s/step - loss: 31952.0156 - val_loss: 31321.9570
Epoch 5/5
20/20 [==============================] - 50s 2s/step - loss: 31952.0156 - val_loss: 31321.9570


In [10]:
#zapisz
autoencoder.save("autoencoder_model.h5")


In [ ]:
#wczytaj
# from tensorflow.keras.models import load_model

# loaded_autoencoder = load_model("autoencoder_model.h5")

In [11]:
#MSE jak dobry jest nasz enkoder

#Odtworzone przez enkoder obrazy dla test
reconstructed_images = autoencoder.predict(x_test)

#MSE dla każdego obrazka
mse_per_image = np.mean((x_test - reconstructed_images) ** 2, axis=(1, 2, 3))

#Avg MSE
average_mse = np.mean(mse_per_image)

print("Average MSE:", average_mse)

193/193 [==============================] - 12s 63ms/step
Average MSE: 31898.174


In [12]:
#Encode dla x_val i x_test
x_val_encoded = encoder.predict(x_val)
x_test_encoded = encoder.predict(x_test)


x_val_encoded = x_val_encoded.reshape(x_val_encoded.shape[0], -1)
x_test_encoded = x_test_encoded.reshape(x_test_encoded.shape[0], -1)

193/193 [==============================] - 5s 25ms/step


In [15]:
#Train Isolation Forest,  Local Outlier Factor, One-Class SVM 
iso_forest_encoded = IsolationForest(contamination=0.16)
iso_forest_encoded.fit(x_test_encoded)

lof_encoded = LocalOutlierFactor(contamination=0.16, novelty=True)
lof_encoded.fit(x_test_encoded)

oc_svm_encoded = OneClassSVM(nu=0.16, kernel='rbf', gamma='scale')
oc_svm_encoded.fit(x_test_encoded)

OneClassSVM(nu=0.16)

In [16]:
#Predictions dla Test
iso_forest_encoded_pred = iso_forest_encoded.predict(x_val_encoded)
lof_encoded_pred = lof_encoded.predict(x_val_encoded)
oc_svm_encoded_pred = oc_svm_encoded.predict(x_val_encoded)

#Połączony model voting
combined_pred = mode(np.vstack((iso_forest_encoded_pred, lof_encoded_pred, oc_svm_encoded_pred)), axis=0)[0].flatten()

C:\Users\HP\AppData\Local\Temp\ipykernel_12720\2103271786.py:7: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  combined_pred = mode(np.vstack((iso_forest_encoded_pred, lof_encoded_pred, oc_svm_encoded_pred)), axis=0)[0].flatten()


In [17]:
#Wyniki
models = [iso_forest_encoded_pred, lof_encoded_pred, oc_svm_encoded_pred, combined_pred]
model_names = ['Isolation Forest', 'Local Outlier Factor', 'One-Class SVM', 'Combined (Voting)']
for name, pred in zip(model_names, models):
    print(f'\n{name} Results:')

    # Convert anomalies to binary labels (1 for normal, -1 for anomaly)
    binary_pred = (pred == -1).astype(int)
    binary_y_test = y_val.flatten()
    
    print('Comparison between detected anomalies and y_test')
      
    print('Recall:', recall_score(binary_y_test,binary_pred))
    print('Precision:', precision_score(binary_y_test,binary_pred))
    print('Accuracy:',accuracy_score(binary_y_test,binary_pred))


Isolation Forest Results:
Comparison between detected anomalies and y_test
Recall: 0.1100244498777506
Precision: 0.1461038961038961
Accuracy: 0.69384765625

Local Outlier Factor Results:
Comparison between detected anomalies and y_test
Recall: 0.10757946210268948
Precision: 0.12188365650969529
Accuracy: 0.6669921875

One-Class SVM Results:
Comparison between detected anomalies and y_test
Recall: 0.08801955990220049
Precision: 0.11428571428571428
Accuracy: 0.681640625

Combined (Voting) Results:
Comparison between detected anomalies and y_test
Recall: 0.08801955990220049
Precision: 0.12631578947368421
Accuracy: 0.6962890625
